# Assignment 1: Source Distribution on an Airfoil



Start the script off by importing the libraries needed and by allowing the plots to show in the Jupyter notebook.

In [1]:
import numpy
import math
from matplotlib import pyplot

%matplotlib inline

Get the strength and location data for the sources used to simulate the NACA0012 airfoil.

In [14]:
x_source = numpy.loadtxt("NACA0012_x.txt")
y_source = numpy.loadtxt("NACA0012_y.txt")
strength_source = numpy.loadtxt("NACA0012_sigma.txt")

Next, create the grid of points that the velocity will be ccalculated at using the same process shown in the lessons.

In [2]:
N = 51
x_start, x_end = -2.0, 1.0
y_start, y_end = -0.5, 0.5
x = numpy.linspace(x_start,x_end,N)
y = numpy.linspace(y_start,y_end,N)
X, Y = numpy.meshgrid(x,y)

In [3]:
x_source = numpy.loadtxt("NACA0012_x.txt")
y_source = numpy.loadtxt("NACA0012_y.txt")
strength_source = numpy.loadtxt("NACA0012_sigma.txt")

In [4]:
u_inf = 1.0

u_freestream = u_inf*numpy.ones((N,N),dtype=float)
v_freestream = numpy.zeros((N,N),dtype=float)

psi_freestream = u_inf*Y

In [5]:
def get_velocity(strength, xs, ys, X, Y):
    
    u = strength/(2*numpy.pi)*(X-xs)/((X-xs)**2+(Y-ys)**2)
    v = strength/(2*numpy.pi)*(Y-ys)/((X-xs)**2+(Y-ys)**2)
    
    return u, v

In [11]:
u_source = 0
v_source = 0

for i in range(numpy.size(strength_source)):
    u, v = get_velocity(strength_source[i],x_source[i],y_source[i],X,Y)
    u_source += u
    v_source += v
    print(u)
    


[[  1.00822814e-04   1.02765877e-04   1.04782795e-04 ...,   1.40191361e-04
    7.27554082e-05  -6.11670213e-07]
 [  1.01037262e-04   1.02993237e-04   1.05024114e-04 ...,   1.51406490e-04
    7.88422082e-05  -6.63627381e-07]
 [  1.01243828e-04   1.03212278e-04   1.05256644e-04 ...,   1.63990837e-04
    8.57214636e-05  -7.22497157e-07]
 ..., 
 [  1.01257972e-04   1.03227278e-04   1.05272570e-04 ...,   1.64927456e-04
    8.62355697e-05  -7.26902991e-07]
 [  1.01051961e-04   1.03008823e-04   1.05040659e-04 ...,   1.52239413e-04
    7.92959092e-05  -6.67505114e-07]
 [  1.00838061e-04   1.02782041e-04   1.04799950e-04 ...,   1.40935070e-04
    7.31577681e-05  -6.15100983e-07]]
[[ -1.34536611e-06  -1.37130409e-06  -1.39822815e-06 ...,  -1.83582109e-06
   -9.35693171e-07   4.06086649e-08]
 [ -1.34823636e-06  -1.37434725e-06  -1.40145825e-06 ...,  -1.98271342e-06
   -1.01390516e-06   4.40513847e-08]
 [ -1.35100137e-06  -1.37727932e-06  -1.40457098e-06 ...,  -2.14755845e-06
   -1.10229443e-06   

In [13]:
numpy.size(u_source)

2601